![Py4Eng](img/logo.png)

# Python .NET interface
## Yoav Ram

In this session we will see how to call .NET (specifically, C#) from Python. Similarly, Python can also be called from .NET, but we will not cover this here.

## Setup 

First, we need to install .NET:

- On Windows, [install the .NET framework](https://msdn.microsoft.com/en-us/library/5a4x27ek.aspx).
- On Mac or Linux, install [mono](http://www.mono-project.com) (on Mac, if you have _Homebrew_, just do `brew install mono`).

Next, install [Python for .NET](http://pythonnet.github.io): `pip install pythonnet`. If you are having issues, try `pip install pycparser` on Linux/Mac, and try to get the latest code from [github](https://github.com/pythonnet/pythonnet) and install with `python setup.py install`.

Now, check that it works - this import should succeed:

In [1]:
import clr

## Simple examples

Let's start with a .NET String object:

In [19]:
from System import String

In [25]:
s = String('Hello Microsoft!')
print(s)

Hello Microsoft!


In [26]:
s.Length

16

In [27]:
s.Contains('A'), s.Contains('H')

(False, True)

We can load a more interesting .NET object:

In [28]:
from System import Environment

In [5]:
Environment.MachineName

'SilverBullet.home'

To load additional namespaces, we use the `clr.AddReference` method:

In [29]:
clr.AddReference('System.Collections');
from System.Collections import Hashtable

In [30]:
hashtable = Hashtable()
hashtable.Add('Python', 6)
hashtable.Add('.NET', 4)

hashtable.ContainsKey('Python'), hashtable['Python']

(True, 6)

## Exercise 1
Use [`StringBuilder`](https://msdn.microsoft.com/en-us/library/system.text.stringbuilder(v=vs.110).aspx) (a mutable version of `String`) to build a string representing a triangle of stars. The bottom side of the triangle should have `n` stars.

For example, for `n=10`, the result should be:

```
    *    
   ***   
  *****  
 ******* 
*********
```

In [30]:
from System.Text import StringBuilder

    *    
   ***   
  *****  
 ******* 
*********



## Example: XML writer

We can use .NET facilities to write an XML writer using the XMLTextWriter class ([source](https://www.thecodingforums.com/threads/example-code-python-pythonnet-ironpython-boo.336422/)). This approach is useful for things that don't exist in Python, are more efficient in .NET, or things we already know how to do in .NET:

In [2]:
import System.Xml
filename = "tmp.xml"

In [5]:
writer = System.Xml.XmlTextWriter(filename, None)
# Use indenting for readability.
writer.Formatting = System.Xml.Formatting.Indented

writer.WriteComment("XML written from Python using pyhtonnet")
writer.WriteStartElement("PythonCourses")

# start course element
writer.WriteStartElement("Course")
writer.WriteStartAttribute("name")
writer.WriteString("Py4Eng")
writer.WriteEndAttribute(); 

# city element
writer.WriteStartElement("city")
writer.WriteString("Rehovot")
writer.WriteEndElement()

# students element
writer.WriteElementString("Students", "20")

# website element
writer.WriteStartElement("website")
writer.WriteString("http://python.yoavram.com")
writer.WriteEndElement()

# end the course element
writer.WriteEndElement()

# end the root element
writer.WriteEndElement()

# write to file and close the writer
writer.Flush()
writer.Close()

In [6]:
%cat tmp.xml

<!--XML written from Python using pyhtonnet-->
<PythonCourses>
  <Course name="Py4Eng">
    <city>Rehovot</city>
    <Students>20</Students>
    <website>http://python.yoavram.com</website>
  </Course>
</PythonCourses>

Now read the file back in and parse to ensure well formed XML:

In [8]:
doc = System.Xml.XmlDocument()
doc.PreserveWhitespace = 1 # Preserve white space for readability.
doc.Load(filename)
print(doc.InnerXml)

<!--XML written from Python using pyhtonnet-->
<PythonCourses>
  <Course name="Py4Eng">
    <city>Rehovot</city>
    <Students>20</Students>
    <website>http://python.yoavram.com</website>
  </Course>
</PythonCourses>


## Example: Load C# library

Next, we will write a C# library and load it to Python.

Start by writing a C# library. We'll write a simple Fibonacci number calculator:

In [19]:
%%file FibonacciExample.cs
using System;

namespace Com.Yoavram.Examples
{
   public class FibonacciExample
   {
      public static int Fibonacci(int n)
      {
          int a = 1;
          int b = 1;
          int i = 2;
          while (i < n) {
              int tmp = b;
              b = a + b;
              a = tmp;
              i++;
          }
          return b;
      }
      public static void Main(string[] args)
      {
         Console.WriteLine("Fibonacci 10:");
         Console.WriteLine(Fibonacci(10));
      }
   } 
}

Overwriting FibonacciExample.cs


Now to compile it and run it on MacOS (running is calling the `Main` function):

In [20]:
!mcs FibonacciExample.cs
!mono FibonacciExample.exe

Fibonacci 10:
55


Now we load the libary to Python and import the namespace:

In [11]:
assembly = clr.AddReference('FibonacciExample')
from Com.Yoavram.Examples import FibonacciExample

so that we can use the functions from the namespace:

In [12]:
FibonacciExample.Fibonacci(10)

55

Let's compete the .NET Fibonacci function with a Python one:

In [14]:
def fibonacci(n):
    a = 1
    b = 1
    i = 2
    while i < n:
        a, b = b, a + b
        i+= 1
    return b

In [15]:
fibonacci(10)

55

In [17]:
%timeit fibonacci(100)
%timeit FibonacciExample.Fibonacci(100)

100000 loops, best of 3: 8.43 µs per loop
The slowest run took 20.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.76 µs per loop


Alright, the .NET implementation is faster!

Now let's write a C# library and wrap it with a Python function. The following code calculates the cumulative sum of a given list/array of integers:

In [24]:
%%file CumSumExample.cs
namespace Com.Yoavram.Examples
{
   public class CumSumExample
   {
       public static int[] CumSum(int[] input)
      {
          int[] output = new int[input.Length];
          if (input.Length > 0) {
              output[0] = input[0];
              for (int i = 1; i < input.Length; i++)
              {
                 output[i] = input[i] + output[i-1];
              }
          }
          return output;
      }
      public static void Main(string[] args)
      {
      }
   } 
}

Overwriting CumSumExample.cs


In [25]:
!mcs CumSumExample.cs

This will be the Python wrapper - it will convert types for us so that we don't even need to know we are using .NET (except for making sure we have .NET and `pythonnet` installed...).

In [26]:
import clr
assembly = clr.AddReference('SumExample')
from Com.Yoavram.Examples import SumExample

def cumsum(data):
    return list(SumExample.Sum(data))

In [28]:
cumsum([1, 2, 3, 4, 5, 6, 7, 8, 9])

[1, 3, 6, 10, 15, 21, 28, 36, 45]

## Exercise 2

Write C# function that given an integer `n`, creates a triangle, such as in Exercise 1, and returns a `String` (not the `StringBuilder`!). Call it from Python.

## Solutions

# Exercise 1

In [51]:
from System.Text import StringBuilder

n = 10
sb = StringBuilder()
for k in range(1, n+1, 2):
    sb.Append(' '*((n-k)//2))
    sb.Append('*'*k)
    sb.Append(' '*((n-k)//2))
    sb.Append('\n')
print(sb)

    *    
   ***   
  *****  
 ******* 
*********



## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com) and is part of the [_Python for Engineers_](https://github.com/yoavram/Py4Eng) course.

The notebook was written using [Python](http://python.org/) 3.6.1.
Dependencies listed in [environment.yml](../environment.yml), full versions in [environment_full.yml](../environment_full.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)